In [1]:
from collections import defaultdict
import nltk
import os
import numpy as np
from nltk.corpus import machado, mac_morpho
from nltk.tokenize import WordPunctTokenizer
from nltk.corpus import stopwords
import string
from collections import defaultdict
from nltk.stem.snowball import PortugueseStemmer
import enchant
from gensim import corpora, models, similarities

In [2]:
textos = []
for p, d, f in os.walk(r'machado/machado'):
    #print( p,d,f)
    if f:
        for fileid  in f:
            if not fileid.endswith('.txt'):
                continue
            with open(os.path.join(p,fileid), encoding='iso-8859-1') as g:
                textos.append(g.read())

In [3]:
swu = stopwords.words('portuguese') + list (string.punctuation)
stemmer = PortugueseStemmer()

In [4]:
#textos = textos[:4]

In [5]:
def clean_sentence(texto : str):
    return [stemmer.stem(token.lower()) for token in WordPunctTokenizer().tokenize(texto) if token not in swu]
textos = [*map(clean_sentence,textos)]

In [6]:
all_words = set()
for text in textos:
    all_words |= set(text)
ordered_words = list(sorted(all_words))
index_word = dict([(word,i) for i, word in enumerate(ordered_words)])

In [7]:
matrix = [[0]*len(textos) for _ in range(len(all_words))]
for i, document in enumerate(textos):
    for word in document:
        j = index_word[word]
        matrix[j][i] += 1
matrix = np.matrix(matrix)
binary_matrix = (matrix >= 1)*1

## Questão 1

In [8]:
def frequency_from(word, matrix):
    clean_word = clean_sentence(word)[0]
    if clean_word not in index_word:
        return [[0]*matrix.shape[1]]
    word_index = index_word[clean_word]
    
    line = matrix[word_index]
    return line/np.sum(line)

def binary_frequency(word):
    return frequency_from(word,binary_matrix)

def word_frequency(word):
    return frequency_from(word,matrix)

In [9]:
binary_frequency('agora')

matrix([[0.00487805, 0.        , 0.        , 0.00487805, 0.00487805,
         0.        , 0.        , 0.        , 0.00487805, 0.        ,
         0.00487805, 0.00487805, 0.        , 0.00487805, 0.        ,
         0.        , 0.00487805, 0.00487805, 0.        , 0.00487805,
         0.00487805, 0.00487805, 0.00487805, 0.00487805, 0.        ,
         0.00487805, 0.00487805, 0.        , 0.        , 0.00487805,
         0.00487805, 0.00487805, 0.00487805, 0.        , 0.        ,
         0.00487805, 0.        , 0.        , 0.00487805, 0.        ,
         0.00487805, 0.        , 0.00487805, 0.00487805, 0.00487805,
         0.        , 0.00487805, 0.00487805, 0.00487805, 0.00487805,
         0.00487805, 0.00487805, 0.00487805, 0.00487805, 0.00487805,
         0.00487805, 0.00487805, 0.00487805, 0.00487805, 0.00487805,
         0.00487805, 0.00487805, 0.00487805, 0.00487805, 0.        ,
         0.        , 0.00487805, 0.00487805, 0.00487805, 0.00487805,
         0.00487805, 0.00487805, 0

In [10]:
word_frequency('agora')

matrix([[0.00070497, 0.        , 0.        , 0.00035249, 0.00140994,
         0.        , 0.        , 0.        , 0.00035249, 0.        ,
         0.00458231, 0.00105746, 0.        , 0.00035249, 0.        ,
         0.        , 0.00070497, 0.00140994, 0.        , 0.00105746,
         0.00140994, 0.00035249, 0.00035249, 0.00035249, 0.        ,
         0.00035249, 0.00070497, 0.        , 0.        , 0.00035249,
         0.00035249, 0.00387734, 0.00105746, 0.        , 0.        ,
         0.00845964, 0.        , 0.        , 0.00140994, 0.        ,
         0.00035249, 0.        , 0.00105746, 0.00176243, 0.00035249,
         0.        , 0.00035249, 0.0024674 , 0.00035249, 0.00105746,
         0.00070497, 0.00070497, 0.01762425, 0.0024674 , 0.00281988,
         0.0024674 , 0.00669722, 0.00352485, 0.00035249, 0.00281988,
         0.00070497, 0.00317237, 0.00176243, 0.00105746, 0.        ,
         0.        , 0.02220656, 0.00140994, 0.00387734, 0.00387734,
         0.00176243, 0.00035249, 0

In [11]:
def tfdf(word):
    clean_word = clean_sentence(word)[0]
    if clean_word not in index_word:
        return [[0]*matrix.shape[1]]
    word_index = index_word[clean_word]
    line = matrix[word_index]
    quantity_words_per_doc = matrix.sum(axis=0)
    tf = line/quantity_words_per_doc
    
    freq_word = ((tf > 0)*1).sum()
    assert freq_word > 0
    df = np.log(matrix.shape[1]/freq_word)
    assert df >= 0
    return tf*df

In [12]:
tfdf('agora')

matrix([[5.87186978e-04, 0.00000000e+00, 0.00000000e+00, 1.67729123e-04,
         1.03152224e-03, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
         1.14956845e-04, 0.00000000e+00, 2.77766347e-04, 5.72138777e-04,
         0.00000000e+00, 3.61749121e-04, 0.00000000e+00, 0.00000000e+00,
         8.53965137e-04, 2.14559055e-04, 0.00000000e+00, 1.26612192e-03,
         2.19202353e-04, 1.10097558e-04, 3.88745324e-04, 3.95491446e-04,
         0.00000000e+00, 3.61749121e-04, 3.92384713e-05, 0.00000000e+00,
         0.00000000e+00, 1.40571748e-04, 3.01357945e-04, 1.08771945e-04,
         1.85663500e-04, 0.00000000e+00, 0.00000000e+00, 2.90783982e-04,
         0.00000000e+00, 0.00000000e+00, 7.97906157e-04, 0.00000000e+00,
         1.64550142e-04, 0.00000000e+00, 2.18961037e-04, 2.73838325e-04,
         1.75140785e-04, 0.00000000e+00, 1.00231752e-04, 4.51930205e-04,
         3.05908652e-04, 1.35387295e-04, 3.93783060e-04, 1.28124776e-04,
         3.56194187e-04, 3.43724993e-04, 3.07326687

## Questão 2

In [13]:
def get_log_normalized(matrix):
    log_matrix = np.log(matrix+1)
    squared_log_matrix = np.square(log_matrix)
    sum_rows = squared_log_matrix.sum(axis=1)
    log_normalized_matrix = np.power(np.multiply(squared_log_matrix,np.power(sum_rows, -1)),0.5)
    return log_normalized_matrix

In [14]:
log_normalized_matrix = get_log_normalized(matrix)
log_normalized_matrix

matrix([[0.        , 0.        , 0.        , ..., 0.        , 0.        ,
         0.        ],
        [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
         0.        ],
        [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
         0.        ],
        ...,
        [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
         0.        ],
        [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
         0.        ],
        [0.        , 0.        , 0.        , ..., 0.07960309, 0.07960309,
         0.        ]])

## Questão 3

In [15]:
dicionario = corpora.Dictionary(textos)
dicionario.save('vocabulario.dict')
dicionario

In [16]:
corpus = [dicionario.doc2bow(d) for d in textos]
corpora.MmCorpus.serialize('machado.mm', corpus)

## Questão 4

In [17]:
lsi = models.LsiModel(corpus, id2word=dicionario, num_topics=4)
index = similarities.MatrixSimilarity(lsi[corpus])

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


In [18]:
sentence = "joao agora aqui"

In [19]:
qt = [stemmer.stem(t) for t in WordPunctTokenizer().tokenize(sentence.lower())]
print(qt)
vec_bow = dicionario.doc2bow(qt)
vec_lsi = lsi[vec_bow] # convert the query to LSI space   
vec_lsi 

['joa', 'agor', 'aqu']


[(0, 0.09771021547692504),
 (1, 0.01822187355319206),
 (2, -0.039713836875367316),
 (3, -0.0876312448720743)]

In [20]:
sims = index[vec_lsi] # perform a similarity query against the corpus
# print(list(enumerate(sims))) # print (document_number, document_similarity) 2-tuples
sims = sorted(enumerate(sims), key=lambda item: -item[1])
print(sims) 

[(37, 0.85971504), (218, 0.8542568), (0, 0.84899867), (2, 0.8213986), (214, 0.81907403), (24, 0.8132994), (4, 0.8097191), (91, 0.80959386), (242, 0.8066621), (167, 0.8036686), (216, 0.801332), (235, 0.79966766), (38, 0.7986653), (222, 0.7966469), (238, 0.79510677), (224, 0.79510605), (194, 0.793702), (42, 0.79231066), (245, 0.79110324), (225, 0.79089516), (30, 0.7895627), (232, 0.7894951), (10, 0.78824335), (223, 0.78064126), (12, 0.779895), (18, 0.77902246), (16, 0.77499115), (14, 0.7715805), (39, 0.7713185), (20, 0.76688164), (237, 0.76447964), (9, 0.76427174), (236, 0.76192117), (233, 0.761158), (212, 0.75837183), (174, 0.7571731), (191, 0.7560776), (197, 0.748665), (40, 0.74842596), (192, 0.7482994), (11, 0.74534684), (23, 0.7448559), (228, 0.7437148), (196, 0.74282235), (6, 0.74109787), (43, 0.74047554), (143, 0.7387937), (44, 0.7379748), (21, 0.7371567), (90, 0.73437077), (27, 0.7335264), (241, 0.73300195), (19, 0.731262), (230, 0.7304253), (7, 0.730322), (13, 0.7256173), (213, 0

In [21]:
lsi.show_topics() 

[(0,
  '0.493*"é" + 0.300*"\x97" + 0.212*"outr" + 0.177*"a" + 0.171*"nã" + 0.160*"o" + 0.147*"tod" + 0.147*"pod" + 0.122*"cas" + 0.114*"diz"'),
 (1,
  '-0.898*"--" + 0.218*"\x97" + -0.183*"diss" + -0.135*"oliv" + 0.114*"é" + -0.104*"sr" + -0.053*"jud" + -0.048*"sik" + -0.047*"respond" + -0.045*"bumbl"'),
 (2,
  '0.701*"\x97" + -0.423*"é" + 0.161*"diss" + 0.158*"..." + 0.108*"mar" + 0.101*"rubiã" + 0.095*"capítul" + 0.094*"cas" + -0.089*"\x93" + 0.084*"mã"'),
 (3,
  '0.402*"gilliatt" + 0.347*"o" + 0.251*"a" + 0.205*"mar" + -0.167*"..." + 0.154*"águ" + 0.150*"lethierry" + 0.141*"vent" + -0.115*"outr" + 0.115*"clubin"')]